# 035-rede2018_codifica_doador_id

In [1]:
ano_eleicao = '2018'

In [2]:
dbschema = f'rede{ano_eleicao}'
dbschema_tse = f'tse{ano_eleicao}'
table_receitas = f'{dbschema}.receitas_{ano_eleicao}'
table_receitas_candidatos = f'{dbschema_tse}.receitas_candidatos_{ano_eleicao}'
table_receitas_candidatos_doador_originario = f'{dbschema_tse}.receitas_candidatos_doador_originario_{ano_eleicao}'
table_receitas_orgaos_partidarios = f'{dbschema_tse}.receitas_orgaos_partidarios_{ano_eleicao}'
table_receitas_orgaos_partidarios_doador_originario= f'{dbschema_tse}.receitas_orgaos_partidarios_doador_originario_{ano_eleicao}'
table_consulta_cand = f'{dbschema_tse}.consulta_cand_{ano_eleicao}'
table_candidaturas = f"{dbschema}.candidaturas_{ano_eleicao}"
table_municipios = f"tse{ano_eleicao}.municipios_{ano_eleicao}"
table_partidos = f'{dbschema_tse}.partidos_{ano_eleicao}'
table_orgaos_partidarios =  f"{dbschema}.orgaos_partidarios_{ano_eleicao}"
table_origem_receitas = f'tse.origem_receitas'
table_fonte_receitas = f'tse.fonte_receitas'

In [3]:
import os
import sys
sys.path.append('../')
import mod_tse as mtse
home = os.environ["HOME"]

In [4]:
mtse.execute_query(f'CREATE SCHEMA IF NOT EXISTS {dbschema};')

In [5]:
table_receptores = f"{dbschema}.receptores_{ano_eleicao}"

query_create_table_receptores = f"""
DROP TABLE IF EXISTS {table_receptores} CASCADE;
CREATE TABLE {table_receptores} (
            receita_origem_sg              varchar,
            receptor_id                    varchar,
            receptor_tipo_cd               varchar,
            receptor_subtipo_cd            varchar,
            receptor_tipo_ds               varchar,
            receptor_subtipo_ds            varchar,
            receptor_label                 varchar,
            receptor_genero_ds             varchar,
            receptor_esfera_partidaria_cd  varchar,
            receptor_esfera_partidaria_ds  varchar,
            receptor_uf                    varchar,
            receptor_ue                    varchar,
            receptor_ue_nome               varchar,
            receptor_cnpj                  varchar,
            receptor_cargo_cd              varchar,
            receptor_cargo_ds              varchar,
            receptor_candidato_seq         varchar,
            receptor_candidato_nr          varchar,
            receptor_candidato_nome        varchar,
            receptor_candidato_cpf         varchar,
            receptor_vice_candidato_cpf    varchar,
            receptor_partido_nr            varchar,
            receptor_partido_sg            varchar,
            receptor_partido_nome          varchar
);

CREATE INDEX ON {table_receptores} (receptor_cnpj);
CREATE INDEX ON {table_receptores} (receptor_candidato_nr);
CREATE INDEX ON {table_receptores} (receptor_uf);
CREATE INDEX ON {table_receptores} (receptor_candidato_cpf);
CREATE INDEX ON {table_receptores} (receita_origem_sg);
CREATE INDEX ON {table_receptores} (receita_origem_sg);
CREATE INDEX ON {table_receptores} (receptor_tipo_cd);
CREATE INDEX ON {table_receptores} (receptor_subtipo_cd);
CREATE INDEX ON {table_receptores} (receptor_tipo_ds);
CREATE INDEX ON {table_receptores} (receptor_subtipo_ds);
CREATE INDEX ON {table_receptores} (receptor_cnpj);


insert into {table_receptores}
    ( select 
            receita_origem_sg,
            receptor_id                        ,
            receptor_tipo_cd                   ,
            receptor_subtipo_cd                ,
            receptor_tipo_ds                   ,
            receptor_subtipo_ds                ,
            receptor_label                     ,
            receptor_genero_ds                 ,
            receptor_esfera_partidaria_cd      ,
            receptor_esfera_partidaria_ds      ,
            receptor_uf                        ,
            receptor_ue                        ,
            receptor_ue_nome                   ,
            receptor_cnpj                      ,
            receptor_cargo_cd                  ,
            receptor_cargo_ds                  ,
            receptor_candidato_seq             ,
            receptor_candidato_nr              ,
            receptor_candidato_nome            ,
            receptor_candidato_cpf             ,
            receptor_vice_candidato_cpf        ,
            receptor_partido_nr                ,
            receptor_partido_sg                ,
            receptor_partido_nome              
      from {table_receitas}
      where 
            tabela_id in ('RC','ROP')
      group by 
            receita_origem_sg,
            receptor_id                        ,
            receptor_tipo_cd                   ,
            receptor_subtipo_cd                ,
            receptor_tipo_ds                   ,
            receptor_subtipo_ds                ,
            receptor_label                     ,
            receptor_genero_ds                 ,
            receptor_esfera_partidaria_cd      ,
            receptor_esfera_partidaria_ds      ,
            receptor_uf                        ,
            receptor_ue                        ,
            receptor_ue_nome                   ,
            receptor_cnpj                      ,
            receptor_cargo_cd                  ,
            receptor_cargo_ds                  ,
            receptor_candidato_seq             ,
            receptor_candidato_nr              ,
            receptor_candidato_nome            ,
            receptor_candidato_cpf             ,
            receptor_vice_candidato_cpf        ,
            receptor_partido_nr                ,
            receptor_partido_sg                ,
            receptor_partido_nome 
    )
;
"""

mtse.execute_query(query_create_table_receptores)

In [6]:
query_update_doador_ID=f"""
       update {table_receitas} as r
            set doador_id = rp.receptor_id,
                doador_tipo_cd = rp.receptor_tipo_cd,
                doador_subtipo_cd = rp.receptor_subtipo_cd,
                doador_tipo_ds = rp.receptor_tipo_ds,
                doador_subtipo_ds = rp.receptor_subtipo_ds,
                doador_label = rp.receptor_label  
            from {table_receptores} as rp
            where 
              r.tabela_id in ('RC','ROP')
            and
              rp.receptor_cnpj = r.doador_cpf_cnpj  
        ;
        """
mtse.execute_query(query_update_doador_ID)

# AJUSTES ESPECIAS

## ATUALIZA DOADOR_ID PARA ROC - INDENTIFICADO

In [7]:
query_update_doador_ROC_INCOMPLETO=f"""
       update {table_receitas} as r
            set doador_id = 'CF'||doador_cpf_cnpj,
                doador_tipo_cd = 'OP',
                doador_subtipo_cd = 'CF',
                doador_tipo_ds = 'Orgão Partidário',
                doador_subtipo_ds = 'Comitê Financeiro'            
            WHERE
                {ano_eleicao} = '2014'
                and
                tabela_id in ('RC','ROP') 
                and 
                receita_origem_sg = 'ROC' 
                and 
                (
                  doador_nome_rfb like '%PARTIDO%'
                  or 
                  doador_nome_rfb like '%COMISSAO%'
                  or
                  doador_nome_rfb like '%COMITE%'
                  or
                  doador_nome_rfb like '%DIRECAO%'
                  or
                  doador_nome_rfb like '%DIRETORIO%'
                  or
                  doador_nome_rfb like '%SOLIDARIEDADE%'
                  or
                  doador_nome_rfb like '%ELEICAO%'   
                  or
                  doador_nome_rfb like '%DEMOCRATAS%'
                  )
        ;
        """
mtse.execute_query(query_update_doador_ROC_INCOMPLETO)

## ATUALIZA DOADOR_ID PARA RPP - INDENTIFICADO

In [8]:
query_update_doador_RPP_INCOMPLETO=f"""
       update {table_receitas} as r
            set doador_id = 'CF'||doador_cpf_cnpj,
                doador_tipo_cd = 'OP',
                doador_subtipo_cd = 'CF',
                doador_tipo_ds = 'Orgão Partidário',
                doador_subtipo_ds = 'Comitê Financeiro' 
            WHERE
                {ano_eleicao} = '2014'
                and
                tabela_id in ('RC','ROP') 
                and 
                receita_origem_sg = 'RPP' 
                and 
                (
                  doador_nome_rfb like '%PARTIDO%'
                  or 
                  doador_nome_rfb like '%COMISSAO%'
                  or
                  doador_nome_rfb like '%COMITE%'
                  or
                  doador_nome_rfb like '%DIRECAO%'
                  or
                  doador_nome_rfb like '%DIRETORIO%'
                  or
                  doador_nome_rfb like '%SOLIDARIEDADE%'
                  or
                  doador_nome_rfb like '%ELEICAO%'   
                  or
                  doador_nome_rfb like '%DEMOCRATAS%' 
                  )
        ;
        """
mtse.execute_query(query_update_doador_RPP_INCOMPLETO)

In [9]:
query_update_doador_RPP_INCOMPLETO=f"""
       update {table_receitas} as r
            set doador_id = 'OP'||doador_cpf_cnpj,
                doador_tipo_cd = 'OP',
                doador_subtipo_cd = doador_esfera_partidaria_cd,
                doador_tipo_ds = 'Orgão Partidário',
                doador_subtipo_ds = doador_esfera_partidaria_ds 
            WHERE
                {ano_eleicao} = '2018'
                and
                tabela_id in ('RC','ROP') 
                and 
                receita_origem_sg = 'RPP' 
                and 
                (
                  doador_nome_rfb like '%PARTIDO%'
                  or 
                  doador_nome_rfb like '%COMISSAO%'
                  or
                  doador_nome_rfb like '%COMITE%'
                  or
                  doador_nome_rfb like '%DIRECAO%'
                  or
                  doador_nome_rfb like '%DIRETORIO%'
                  )
        ;
        """
mtse.execute_query(query_update_doador_RPP_INCOMPLETO)

In [10]:
query_update_doador_RPP_CA_INCOMPLETO=f"""
       update {table_receitas} as r
            set doador_id = 'OP'||doador_cpf_cnpj,
                doador_tipo_cd = 'OP',
                doador_subtipo_cd = doador_esfera_partidaria_cd,
                doador_tipo_ds = 'Orgão Partidário',
                doador_subtipo_ds = doador_esfera_partidaria_ds 
            WHERE
                {ano_eleicao} = '2018'
                and
                doador_tipo_cd = 'PJ' 
                and receptor_tipo_cd= 'CA' 
                and receita_origem_sg = 'RPP'
                and doador_nome_rfb like '%'||doador_partido_sg||'%'
        ;
        """
mtse.execute_query(query_update_doador_RPP_CA_INCOMPLETO)

In [11]:
query_update_doador_ROC_CA_INCOMPLETO=f"""
       update {table_receitas} as r
            set doador_id = ca.candidato_id,
                doador_tipo_cd = 'CA',
                doador_subtipo_cd = 'CA',
                doador_tipo_ds = 'Candidato',
                doador_subtipo_ds = 'Candidato',
                doador_label =  ca.candidato_label
            from {table_candidaturas} ca
            WHERE
                {ano_eleicao} = '2018'
                and
                tabela_id in ('RC','ROP')
                and
                doador_tipo_cd = 'PJ' and receptor_tipo_cd= 'CA' 
                and receita_origem_sg = 'ROC'
                and doador_nome like ca.nm_candidato
        ;
        """
mtse.execute_query(query_update_doador_ROC_CA_INCOMPLETO)

In [12]:
query_update_doador_ROC_CA_INCOMPLETO=f"""
       update {table_receitas} as r
            set doador_id = ca.candidato_id,
                doador_tipo_cd = 'CA',
                doador_subtipo_cd = 'CA',
                doador_tipo_ds = 'Candidato',
                doador_subtipo_ds = 'Candidato',
                doador_label =  ca.candidato_label
            from {table_candidaturas} ca
            WHERE
                {ano_eleicao} = '2018'
                and
                tabela_id in ('RC','ROP')
                and
                doador_tipo_cd = 'PJ' 
                and receptor_tipo_cd= 'CA' 
                and receita_origem_sg = 'ROC'
                and doador_nome_rfb like '%ELEICAO%'
                and doador_nome_rfb like '%'||ca.nm_candidato||'%'
        ;
        """

mtse.execute_query(query_update_doador_ROC_CA_INCOMPLETO)

## ATUALIZA DOADOR_ID PARA RP - PF

In [13]:
query_update_doador_RP_PF=f"""

       update {table_receitas} as r
            set doador_id = ca.receptor_id||'RP',
                doador_tipo_cd = 'CARP',
                doador_subtipo_cd = 'PF',
                doador_tipo_ds = 'Candidato Pessoa Física',
                doador_subtipo_ds = 'Recursos de pessoas físicas',
                doador_label = ca.receptor_label||' (Pessoa Fisica)'
            from {table_receptores} as ca
            WHERE
                r.tabela_id in ('RC','ROP') 
                and
                r.doador_tipo_cd = 'PF'
                and 
                r.receita_origem_sg = 'RP' 
                and
                r.doador_cpf_cnpj = ca.receptor_candidato_cpf
        ;


       update {table_receitas} as r
            set doador_id = ca.candidato_id||'RP',
                doador_tipo_cd = 'CARP',
                doador_subtipo_cd = 'PF',
                doador_tipo_ds = 'Candidato Pessoa Física',
                doador_subtipo_ds = 'Recursos de pessoas físicas',
                doador_label = ca.candidato_label||' (Pessoa Fisica)'
            from {table_candidaturas} as ca
            WHERE
                r.tabela_id in ('RC','ROP')
                and
                r.doador_tipo_cd = 'PF'
                and 
                r.receita_origem_sg = 'RP' 
                and
                r.doador_cpf_cnpj = ca.nr_cpf_candidato
        ;
        """
mtse.execute_query(query_update_doador_RP_PF)

## AJUSTA ID DE  RONI e RAF

In [14]:
query_update_tipo_doador_RONI_RAF = f"""
update {table_receitas} as r
    set doador_id = receita_origem_sg,
        doador_tipo_cd = receita_origem_sg,
        doador_tipo_ds = receita_origem_ds,
        doador_subtipo_cd = receita_origem_sg,
        doador_subtipo_ds = receita_origem_ds 
from  {table_origem_receitas} ro
where
        doador_id = 'RONI'
        and
        receita_origem_sg in ('RONI','RAF')

;
"""
mtse.execute_query(query_update_tipo_doador_RONI_RAF)

## AJUSTA ID DE RFC

In [15]:
query_update_tipo_doador_RFC = f"""
update {table_receitas} as r
    set doador_id = 'RFC'||doador_id,
        doador_tipo_cd = receita_origem_sg,
        doador_tipo_ds = receita_origem_ds,
        doador_subtipo_cd = receita_origem_sg,
        doador_subtipo_ds = receita_origem_ds 
where
    doador_id = 'PJ'
    and receita_origem_sg in ('RFC')
;
"""
mtse.execute_query(query_update_tipo_doador_RFC)